# 04 - Extract Durban and Medellin

This notebook extracts metrics for landslides in Durban and Medellin. It's similar to 03_ExtractGSDR, but processes precipitation data for Durban from the South African Weather Service and for Medellin from the Colombian Instituto de Hidrología, Meteorología y Estudios Ambientales (IDEAM).  

The outputs of this notebook are: 

- lsdata_durban_rain.csv
- annual_block_maxima_durban.csv
- lsdata_medellin_rain.csv
- annual_block_maxima_medellin.csv

These outputs are read into 05_CombinePrep

**Data required**

Processed data: ls_urban_ts_rf_u.pkl

Original data: 

*Durban*

These data were obtained from the South African Weather Service and used with permission for this study
- durban_hourly_rainfall_data.xls 
- durban_station_locations.xls 

*Medellin* 

Data for Medellín were obtained from the Colombian Instituto de Hidrología, Meteorología y Estudios Ambientales (IDEAM), were used with permission for this study, and are available at http://dhime.ideam.gov.co/atencionciudadano/.
- stations and hourly rainfall data from IDEAM.  



In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pyproj
import shapely
from shapely.ops import transform
import datetime
import warnings
import zipfile
import matplotlib.pyplot as plt
from timezonefinder import TimezoneFinder

In [ ]:
datadir = ''

resultsdir = ''

landslides = pd.read_pickle(resultsdir + 'ls_urban_ts_rf_u.pkl')

### Read and process rainfall data from Durban

In [ ]:
for i in range(6):
    sheet = "Sheet" + str(i+1)
    print(sheet)
    
    tempdf = pd.read_excel("durban_hourly_rainfall_data.xls", 
                        sheet_name = sheet, 
                        header = 0, 
                        usecols = "A:C")

    
    if i == 0: 
        durban_rain = tempdf.copy()
        
    else:
        durban_rain = pd.concat([durban_rain, 
                               tempdf], 
                               axis = 0)

In [ ]:
durban_rain = durban_rain.set_index(pd.to_datetime(durban_rain['Date']))

In [ ]:
#Read names and locations of Durban stations 

durban_stations = pd.read_excel("durban_station_locations.xlsx", 
                        sheet_name = "Sheet1", 
                        header = 0, 
                        usecols = "A:D")

In [ ]:
#convert to a geopandas dataframe
durban_stations = gpd.GeoDataFrame(durban_stations,
                                 geometry=gpd.points_from_xy(durban_stations.longitude, durban_stations.latitude),
                                 crs = "EPSG:4326")


Convert local timestamp to UTC

In [ ]:
#initiate timezone finder
tf = TimezoneFinder()

stz = tf.timezone_at(lng=durban_stations.iloc[0].longitude, lat=durban_stations.iloc[0].latitude)

#first, localize to local time zone
durban_rain = durban_rain.tz_localize(stz, 
                        ambiguous=np.zeros(len(durban_rain), dtype = bool), 
                        nonexistent ='shift_forward')
#then, convert to UTC

durban_rain = durban_rain.tz_convert('UTC')

Get Durban landslides

In [ ]:
durban_landslides = landslides[landslides['UC_NM_MN'] == 'Durban'].copy()

functions for Durban

In [ ]:
#write a function 

def get_durban_stations(lspt, 
                durban_stations,
                buffer_dist = 25000):
    
    
    """
    Function to get the all of the South African Weather Service stations 
    within a defined buffer distance to a landslide point
    
    lspt = a landslide point (with a DATE and spatial location in WGS84)
    durban_stations = pandas dataframe Station_Name and a geometry in WGS84
    buffer_dist = buffer distance in meters
    
    """
   

    wgs84 = pyproj.CRS('EPSG:4326')

    #define azimuthal equidistant crs centered on the landslide

    aeqd = pyproj.Proj(proj='aeqd', ellps='WGS84', datum='WGS84', lat_0=lspt.geometry.y, lon_0=lspt.geometry.x).srs

    #reproject the landslide point

    project = pyproj.Transformer.from_crs(wgs84, aeqd, always_xy=True).transform
    lspt_a = transform(project, lspt.geometry)


    #buffer the landlide point
    lspt_ab = lspt_a.buffer(buffer_dist)

    #reproject the station data 
    ga = durban_stations.to_crs(aeqd)

    #get the indices of all stations within 25 km of the landslide point
    sidx = ga.sindex.query(lspt_ab, predicate = 'intersects')
    
    #columns of the metadata dataframe to use as dictionary keys
    
    gscols = durban_stations.columns
    
    #check if it's empty
    
    if len(sidx) == 0: 
        
        gageinfo = dict(zip(gscols, [None]*len(gscols)))
        
        gageinfo['flag'] = 'no close gages'
        
        gageinfodf = pd.DataFrame([gageinfo])
        
    else: 
        
        #continue

        #these are the stations within the buffer distance of the landslide point
        gs = ga.iloc[sidx].copy()

        #get the distance from the stations to the landslide point
        dsl = gs.distance(lspt_a)

        #record the distance
        gs.loc[dsl.index, 'station_dist'] = dsl.values 

        #project stations back to WGS84

        gs = gs.to_crs(wgs84)

        #return the info for the nearest station 

        gageinfo = gs

        gageinfo['flag'] = 'coverage'

        gageinfodf = gageinfo.copy()
            
    gageinfodf['lsidx'] = lspt.name
                
    return gageinfodf
    

In [ ]:
def prep_ts_durban(lspt, durban_rain, daysbefore = 90, daysafter = 1):
    
    """
    Function to interpolate missing values, and subset to the specified time interval before
    and after the landslide point
    
    lspt = row of a dataframe that contains columns: 
        - 'Folder' - folder containing the data (e.g. US.zip)
        - 'date_local_midnight_utc' - midnight local time on the day the landslide occurred, converted to UTC
    
    durban_rain = pandas dataframe of durban rain
    daysbefore = how many days before the landslide should data be extracted?
    daysafter = how many days after midnight on the day of the landslide should be extracted? 
    
    Returns: 
    ts_sub = original time series, subset
    ts_fill = original time series, subset and interpolated   
    
    """
    
    #get the time series from the correct station
    ts = durban_rain[durban_rain['Station_Name'] == lspt['station_name']]['Rain']

    #subset to specified number of days before and after the landslide
    lsdate = lspt['date_local_midnight_utc']

    #check that these dates are in the range of the time series

    start_ts = lsdate - datetime.timedelta(days = daysbefore)
    end_ts = lsdate + datetime.timedelta(days = daysafter + 1)


    ts_sub =  ts.loc[(lsdate - datetime.timedelta(days = daysbefore)):(lsdate + datetime.timedelta(days = daysafter + 1))]

    #sort index, just in case for some reason it wasn't already in sorted, or pandas thinks it isn't
    ts_sub.sort_index(axis = 0, inplace = True)

    #interpolate to fill nans in the time series using the pandas built in interpolation method


    ts_fill = ts_sub.interpolate(method = 'time')

    #remove any duplicates by keeping the first duplicated entry only
    
    ts_fill = ts_fill[~ts_fill.index.duplicated()]
        

    return ts_sub, ts_fill

In [ ]:
def triggering_rain(nhdry, ts, ts_raw, lsdate): 
    
    """
    Gets length and sum of triggering rainfall event from a dry period of length nhdry to the peak hourly
    rainfall and the end of the landslide day
    
    nhdry = length of dry period preceeding landslide in hours
    ts = time series with hourly datetime index and values of hourly precip
    ts_raw = original time series to check how many nans were interpolated in the time range
    lsdate = datetime corresponding to the start of the landslide day  
    
    """
    
    #if the time series is completely empty
    
    if len(ts) == 0:
        
        idxtrst = np.nan
        htrsttopk = np.nan
        cptrsttopk = np.nan
        htrsttoeod = np.nan
        cptrsttoeod = np.nan
        mwtopk = np.nan
        mxhr = np.nan
        idxmxhr = np.nan
        nnantopk = np.nan
        nnantoeod = np.nan
    
    else: 
        
        #find peak hourly rain on the day of the landslide 

        mxhr = ts.loc[lsdate:lsdate+datetime.timedelta(hours = 24)].max()
       

        #if peak hourly rain is nan (e.g. there's no data on the day of the landslide, then)

        if pd.isnull(mxhr):
            
            
            idxmxhr = np.nan
            mxhr = np.nan
            idxtrst = np.nan
            htrsttopk = np.nan
            cptrsttopk = np.nan
            htrsttoeod = np.nan
            cptrsttoeod = np.nan
            mwtopk = np.nan
            nnantopk = np.nan
            nnantoeod = np.nan
            

        else:
            
            #find out when the peak occurred
            idxmxhr = ts.loc[lsdate:lsdate+datetime.timedelta(hours = 24)].idxmax()


            #moving window summing, index is last observation in window (eg. 10 am is 8am + 9am + 10 am)
            mw = ts.rolling(nhdry).sum() 

            #subset moving window time series up to the peak 
            mwtopk = mw.loc[:idxmxhr]

            # find where the nhdry hourly sum is less than 0.01 mm (essentially 0, but catch very small values)
            #, find the nearest index to the peak point (iloc type index)
            ix = mwtopk[mwtopk<0.01].index.get_indexer([idxmxhr], method = 'pad')[0]

            #if we can't find a dry enough period in the time series:
            if ix == -1:

                idxtrst = np.nan
                htrsttopk = np.nan
                cptrsttopk = np.nan
                htrsttoeod = np.nan
                cptrsttoeod = np.nan
                mwtopk = np.nan
                nnantopk = np.nan
                nnantoeod = np.nan
                

            else:

                #get the time stamp index of the start of the triggering rainfall 
                idxtrst = mwtopk[mwtopk<0.01].index[ix] + datetime.timedelta(hours = 1)

                #how long did it rain between the start of the triggering rain and the peak (inclusive)? 

                htrsttopk = ts.loc[idxtrst:idxmxhr].count()

                #how much did it rain between the start of the triggering rain and the peak (inclusive)?
                cptrsttopk = ts.loc[idxtrst:idxmxhr].sum()

                #how long did it rain between the start of the triggering rain and the end of the landslide day? 

                htrsttoeod = ts.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].count()

                #how much did it rain between the start of the triggering rain and the end of the landslide day?
                cptrsttoeod = ts.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].sum()
                
                
                #check what percent of the original time series leading up to the peak is nans
                
                nnantopk = ts_raw.loc[idxtrst:idxmxhr].isna().sum()
                
                #check what percent of the original time series leading up to the end of day is nans
                
                nnantoeod = ts_raw.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].isna().sum()
                             
                
                
                

    return [idxmxhr, 
            mxhr, 
            idxtrst, 
            htrsttopk, 
            cptrsttopk, 
            htrsttoeod, 
            cptrsttoeod, 
            mwtopk, 
            nnantopk,
            nnantoeod]



In [ ]:
def get_antecedent(ts, idxdt, h):
    
    """
    get cumulative antecedent precipitation before some hour (e.g. peak precip on landslide day 
    or before start of triggering rainfall)
    
    ts = time series
    idxdt = datetime that you want antecedent for
    h = number of antecedent hours
    
    """
    #for the case that there no start to the triggering rainfall was found
    if pd.isnull(idxdt):
        ante = np.nan
        
    #for the case that the time series is empty
    elif len(ts) == 0:
        ante = np.nan
    
    else:
        
        ante = ts.loc[idxdt - datetime.timedelta(hours = h):idxdt].sum()
    
    return ante

In [ ]:
def get_rainmetrics_durban(lspt, durban_rain):
    
    """
    get info about triggering, event, and antecedent rainfall for a landslide
    
    lspt - a dataframe row from lsdata
    
    """
    print(lspt.name)
    
    [ts, ts_fill] = prep_ts_durban(lspt, durban_rain, daysbefore = 90, daysafter = 1)
    
    if ts is None: 
        
        rd = None
        
    else:
    
        lsdate = lspt['date_local_midnight_utc']

        #then, get triggering rainfall metrics

        #define triggering event as 3 hours dry to peak or end of day
        [idxmxhr, mxhr, idxtrst, 
         htrsttopk, cptrsttopk, htrsttoeod, cptrsttoeod, _, nnantrsttopk, nnantrsttoeod] = triggering_rain(nhdry = 3,
                                                                                                   ts = ts_fill,
                                                                                                    ts_raw = ts,
                                                                                                   lsdate = lsdate)

        #get event metrics 

        #define event as 48 hours dry to peak or end of day
        [_, _, idxest, 
         hesttopk, cpesttopk, hesttoeod, cpesttoeod, _, nnanesttopk, nnanesttoeod] = triggering_rain(nhdry = 48,
                                                                                ts = ts_fill,
                                                                                ts_raw = ts,
                                                                                lsdate = lsdate)


        #get antecedent metrics 

        #24 hours before start of triggering rain 

        ante24htrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24)

        #7 day before start of triggering rain 
        ante7dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*7)

        #14 day before start of triggering rain 
        ante14dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*14)

        #21 day before start of triggering rain 
        ante21dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*21)

        #28 day before start of triggering rain
        ante28dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*28)


        #get estimated exceedance probabilities of the hourly maximum intensity

       # mxhrexprob = get_exceedanceprob(ts, mxhr, 10) #minimum 10 years on record


        #put everything into a dictionary to return 


        rd = {"event_id":lspt.name, 
             "tr_start":idxtrst,
             "tr_tpk":idxmxhr,
             "tr_ppk":mxhr,
             "tr_htopk":htrsttopk, 
             "tr_cptopk":cptrsttopk, 
             "tr_htoeod":htrsttoeod, 
             "tr_cptoeod":cptrsttoeod,
             "tr_nnantopk": nnantrsttopk, 
             "tr_nnan_toeod": nnantrsttoeod,
             "e_start":idxest, 
             "e_htopk":hesttopk, 
             "e_cptopk":cpesttopk, 
             "e_htoeod":hesttoeod, 
             "e_cptoeod":cpesttoeod,
             "e_nnantopk":nnanesttopk,
             "e_nnantoeod":nnanesttoeod,               
             "tr_ante24h":ante24htrst, 
             "tr_ante7d":ante7dtrst, 
             "tr_ante14d":ante14dtrst, 
             "tr_ante21d":ante21dtrst, 
             "tr_ante28d":ante28dtrst}

    print(lspt.name)

    return rd

Get the gauges within a 25 km radius of the landslide

In [ ]:
for l in range(len(durban_landslides)):
       
    lspt = durban_landslides.iloc[l]
    
    tempdf = get_durban_stations(lspt, 
                durban_stations,
                buffer_dist = 25000)
    
    if l == 0:
        
        gageinfo = tempdf.copy()
        
    else:
        
        gageinfo = pd.concat([gageinfo, tempdf], axis = 0)
    
    print('end {}/{}'.format(l, len(durban_landslides)))

In [ ]:
#join to landslides dataframe (right join)


durban_ls_gages = durban_landslides.merge(gageinfo, 
                           how = 'right',
                            left_index = True,
                           right_on = 'lsidx')


get rain metrics for each landslide/gauge combination

In [ ]:
with warnings.catch_warnings(record = True):
    durban_ls_gages['rain_metrics'] = durban_ls_gages.apply(lambda lspt:get_rainmetrics_durban(lspt, durban_rain), 
                                             axis = 1)

In [ ]:
joindf = pd.DataFrame(list(durban_ls_gages['rain_metrics'].values))

lsdata_durban_rain = pd.concat([durban_ls_gages, 
             joindf.set_index(durban_ls_gages.index)], 
             axis = 1)

lsdata_durban_rain.drop(['rain_metrics'], axis = 1, inplace = True)


#strip city names of white space, special characters, etc for R

lsdata_durban_rain['city'] = lsdata_durban_rain.apply(lambda row:''.join(e for e in row['UC_NM_MN'] if e.isalnum()), 
                                       axis = 1)



In [ ]:
#rename some columns to align with gsdr format to be able to concatenate later

lsdata_durban_rain.rename(columns = {'latitude':'Latitude', 
                                     'longitude':'Longitude',
                                     'station_name':'OriginalID'}, 
                           inplace = True)


#add some fields of nans to be able to concatenate to gsdr later

lsdata_durban_rain['Folder'] = np.nan 
lsdata_durban_rain['NewID'] = np.nan
lsdata_durban_rain['Recordlength(hours)'] = np.nan
lsdata_durban_rain['Recordlength(years)'] = np.nan
lsdata_durban_rain['StartDate'] = np.nan
lsdata_durban_rain['EndDate'] = np.nan
lsdata_durban_rain['Missingdata(%)'] = np.nan
lsdata_durban_rain['coverage'] = np.nan
lsdata_durban_rain['precip_source'] = 'SAWS'



In [ ]:
lsdata_durban_rain_save = lsdata_durban_rain.loc[:,['inventory', 'src_index', 'inventory_id', 'inventory_id_name', 'lsidx',
       'ID_HDC_G0', 'UC_NM_MN', 'date_local_midnight_utc', 'Folder',
       'OriginalID', 'NewID', 'Latitude', 'Longitude', 'Recordlength(hours)',
       'Recordlength(years)', 'StartDate', 'EndDate', 'Missingdata(%)',
       'geometry_y', 'flag', 'coverage', 'station_dist', 'event_id',
       'tr_start', 'tr_tpk', 'tr_ppk', 'tr_htopk', 'tr_cptopk', 'tr_htoeod',
       'tr_cptoeod', 'tr_nnantopk', 'tr_nnan_toeod', 'e_start', 'e_htopk',
       'e_cptopk', 'e_htoeod', 'e_cptoeod', 'e_nnantopk', 'e_nnantoeod',
       'tr_ante24h', 'tr_ante7d', 'tr_ante14d', 'tr_ante21d', 'tr_ante28d',
       'city', 'precip_source']]

In [ ]:
lsdata_durban_rain_save.to_csv(resultsdir + 'lsdata_durban_rain.csv')

In [ ]:
lsdata_durban_rain_save = pd.read_csv(resultsdir + 'lsdata_durban_rain.csv')

### get annual maxima for Durban

In [ ]:
def prep_whole_ts_durban(station):
    
    """
    Function to read the precipitation time series from a gauge, convert it to UTC,
    interpolate missing values
    
    station = row of a dataframe that contains columns: 
        - 'Folder' - folder containing the data (e.g. US.zip)
        - 'OriginalID' - name of station
    gsdrdir = directory containing the GSDR subfolders
    
    
    Returns: 
    ts = original time series
    ts_fill = original time series, interpolated   
    
    """
            
    ts = durban_rain[durban_rain['Station_Name'] == station]['Rain']
    

    #already in UTC

    #sort index, just in case for some reason it wasn't already in sorted, or pandas thinks it isn't
    ts.sort_index(axis = 0, inplace = True)
    
    #interpolate to fill nans in the time series using the pandas built in interpolation method

    ts_fill = ts.interpolate(method = 'time')

    #remove any duplicates by keeping the first duplicated entry only
    
    ts_fill = ts_fill[~ts_fill.index.duplicated()]

    return ts, ts_fill

In [ ]:
durations = [1, 3, 6, 12, 24, 48, 100, 200, 500, 1000] #durations to extract block maxima for

In [ ]:
with warnings.catch_warnings(record = True):
#loop through all stations associated with a landslide and extract annual block maxima at a range of durations

    #loop over stations

    for s in range(len(uniquestations)):
        
        print(s)
        
        station = lsdata_durban_rain_save[lsdata_durban_rain_save['OriginalID'] == uniquestations[s]].iloc[0][['ID_HDC_G0', 
                                                                                          'UC_NM_MN', 
                                                                                          'Folder', 
                                                                                          'OriginalID', 
                                                                                           'NewID', 
                                                                                          'Latitude',
                                                                                          'Longitude',
                                                                                          'Recordlength(hours)',
                                                                                          'Recordlength(years)',
                                                                                          'StartDate',
                                                                                          'EndDate',
                                                                                          'Missingdata(%)',
                                                                                          'geometry_y']]
        

        #get time series

        ts, ts_fill = prep_whole_ts_durban(uniquestations[s])

        #extract block maxima at a range of durations

        for d in range(len(durations)):

           #moving window on the raw time series
            mw = ts.rolling(durations[d]).sum() #right index is sum of previous d observations. 
            #if there are nans within the window, returns nan

            ann_block_max = mw.resample('Y').max() #take annual max. ignores nans (could be missing data, 
            #will still get max)


            #moving window on the interpolated time series

            mw_fill = ts_fill.rolling(durations[d]).sum() #right index is sum of previous d observations. 
            #if there are nans within the window, returns nan

            ann_block_max_fill = mw_fill.resample('Y').max() #take annual max. ignores nans (could be missing data, 
            #will still get max)
            
            block_max_df = pd.DataFrame(ann_block_max.values, index = ann_block_max.index, columns = ['raw_block_max'])
            #block_max_df = pd.DataFrame(ann_block_max, columns = ['raw_block_max'])
            block_max_df['fill_block_max'] = ann_block_max_fill.values

            #record the number of non-nan observations in the moving window (all other hours in the year are then nan)
            block_max_df['raw_notnainmw'] = mw.resample('Y').count().values

            #record the number of non-nan observations in the raw time series (all other hours in the year are then nan)
            block_max_df['raw_notnaints'] = ts.resample('Y').count().values


            block_max_df['duration(h)'] = durations[d]
            block_max_df['year'] = block_max_df.index.year


            if d == 0:

                stationdf = block_max_df.copy() 



            else:

                stationdf = pd.concat([stationdf, block_max_df])


        #assign information about the station
        stationinfo = pd.DataFrame([station])

        stationinfo = stationinfo.loc[stationinfo.index.repeat(len(stationdf))]

        stationinfo.set_index(stationdf.index, inplace = True)

        stationdf = pd.concat([stationinfo, stationdf], axis = 1)

        if s == 0: 

            maindf = stationdf.copy()

        else: 

            maindf = pd.concat([maindf, stationdf])
            
        print(s)

In [ ]:
maindf.to_csv(resultsdir + 'annual_block_maxima_durban.csv')

### Read and process rainfall data from Medellin

In [ ]:
medellin_stations = pd.read_csv("Stations_IDEAM_Hourly.csv")



In [ ]:
medellin_stations = gpd.GeoDataFrame(medellin_stations,
                                 geometry=gpd.points_from_xy(medellin_stations.longitud, medellin_stations.latitud),
                                 crs = "EPSG:4326")

Merge IDEAM data into one pandas dataframe

In [ ]:
ideam_dir = '../Rainfall_IDEAM/Hourly/'

In [ ]:
for i in range(len(medellin_stations)):
    
    fn = ideam_dir + str(medellin_stations.iloc[i]['CODIGO']) + '.csv'
    print(fn)
    
    tempdf = pd.read_csv(fn)
    
    if i == 0: 
        medellin_rain = tempdf.copy()
        
    else:
        medellin_rain = pd.concat([medellin_rain, 
                               tempdf], 
                               axis = 0)
    


In [ ]:
medellin_rain = medellin_rain.set_index(pd.to_datetime(medellin_rain['Fecha'], dayfirst = True))

Convert local time stamps to UTC

In [ ]:
#initiate timezone finder
tf = TimezoneFinder()

stz = tf.timezone_at(lng=medellin_rain.iloc[0].Longitud, lat=medellin_rain.iloc[0].Latitud)

#first, localize to local time zone
medellin_rain = medellin_rain.tz_localize(stz, 
                        ambiguous=np.zeros(len(medellin_rain), dtype = bool), 
                        nonexistent ='shift_forward')
#then, convert to UTC

medellin_rain = medellin_rain.tz_convert('UTC')

get Medellin landslides

In [ ]:
medellin_landslides = landslides[landslides['UC_NM_MN'] == 'Medellín'].copy()

Functions for Medellin

In [ ]:
#write a function 

def get_medellin_stations(lspt, 
                medellin_stations,
                buffer_dist = 25000):
    
    
    """
    Function to get the all of the IDEAM stations 
    within a defined buffer distance to a landslide point
    
    lspt = a landslide point (with a DATE and spatial location in WGS84)
    medellin_stations = pandas dataframe with a geometry in WGS84
    buffer_dist = buffer distance in meters
    
    """
   

    wgs84 = pyproj.CRS('EPSG:4326')

    #define azimuthal equidistant crs centered on the landslide

    aeqd = pyproj.Proj(proj='aeqd', ellps='WGS84', datum='WGS84', lat_0=lspt.geometry.y, lon_0=lspt.geometry.x).srs

    #reproject the landslide point

    project = pyproj.Transformer.from_crs(wgs84, aeqd, always_xy=True).transform
    lspt_a = transform(project, lspt.geometry)


    #buffer the landlide point
    lspt_ab = lspt_a.buffer(buffer_dist)

    #reproject the station data 
    ga = medellin_stations.to_crs(aeqd)

    #get the indices of all stations within 25 km of the landslide point
    sidx = ga.sindex.query(lspt_ab, predicate = 'intersects')
    
    #columns of the metadata dataframe to use as dictionary keys
    
    gscols = medellin_stations.columns
    
    #check if it's empty
    
    if len(sidx) == 0: 
        
        gageinfo = dict(zip(gscols, [None]*len(gscols)))
        
        gageinfo['flag'] = 'no close gages'
        
        gageinfodf = pd.DataFrame([gageinfo])
        
    else: 
        
        #continue

        #these are the stations within the buffer distance of the landslide point
        gs = ga.iloc[sidx].copy()

        #get the distance from the stations to the landslide point
        dsl = gs.distance(lspt_a)

        #record the distance
        gs.loc[dsl.index, 'station_dist'] = dsl.values 

        #project stations back to WGS84

        gs = gs.to_crs(wgs84)

        #return the info for the nearest station 

        gageinfo = gs

        gageinfo['flag'] = 'coverage'

        gageinfodf = gageinfo.copy()
            
    gageinfodf['lsidx'] = lspt.name
                
    return gageinfodf
    

In [ ]:
def prep_ts_medellin(lspt, medellin_rain, daysbefore = 90, daysafter = 1):
    
    """
    Function to interpolate missing values, and subset to the specified time interval before
    and after the landslide point
    
    lspt = row of a dataframe that contains columns: 
        - CODIGO - station name
        - Valor - precipitation value
        - 'date_local_midnight_utc' - midnight local time on the day the landslide occurred, converted to UTC
    
    medellin_rain = pandas dataframe of medellin rain
    daysbefore = how many days before the landslide should data be extracted?
    daysafter = how many days after midnight on the day of the landslide should be extracted? 
    
    Returns: 
    ts_sub = original time series, subset
    ts_fill = original time series, subset and interpolated   
    
    """
    
    #get the time series from the correct station
    ts = medellin_rain[medellin_rain['CodigoEstacion'] == lspt['CODIGO']]['Valor'].copy()
    
    
    #sort index
    ts = ts.sort_index()

    #subset to specified number of days before and after the landslide
    lsdate = lspt['date_local_midnight_utc']

    #check that these dates are in the range of the time series

    start_ts = lsdate - datetime.timedelta(days = daysbefore)
    end_ts = lsdate + datetime.timedelta(days = daysafter + 1)
    
    if (start_ts > ts.index[0]) & (end_ts < ts.index[-1]):

        ts_sub =  ts.loc[(lsdate - datetime.timedelta(days = daysbefore)):(lsdate + datetime.timedelta(days = daysafter + 1))]

        #sort index, just in case for some reason it wasn't already in sorted, or pandas thinks it isn't
        ts_sub.sort_index(axis = 0, inplace = True)

        #interpolate to fill nans in the time series using the pandas built in interpolation method


        ts_fill = ts_sub.interpolate(method = 'time')

        #remove any duplicates by keeping the first duplicated entry only
        ts_fill = ts_fill[~ts_fill.index.duplicated()]
    
    else: #case that the time series can't be extracted because time series is not completely covered
        ts_sub = np.array([])
        ts_fill = np.array([])
        

    return ts_sub, ts_fill

In [ ]:
def triggering_rain(nhdry, ts, ts_raw, lsdate): 
    
    """
    Gets length and sum of triggering rainfall event from a dry period of length nhdry to the peak hourly
    rainfall and the end of the landslide day
    
    nhdry = length of dry period preceeding landslide in hours
    ts = time series with hourly datetime index and values of hourly precip
    ts_raw = original time series to check how many nans were interpolated in the time range
    lsdate = datetime corresponding to the start of the landslide day  
    
    """
    
    #if the time series is completely empty
    
    if len(ts) == 0:
        
        idxtrst = np.nan
        htrsttopk = np.nan
        cptrsttopk = np.nan
        htrsttoeod = np.nan
        cptrsttoeod = np.nan
        mwtopk = np.nan
        mxhr = np.nan
        idxmxhr = np.nan
        nnantopk = np.nan
        nnantoeod = np.nan
    
    else: 
        
        #find peak hourly rain on the day of the landslide 

        mxhr = ts.loc[lsdate:lsdate+datetime.timedelta(hours = 24)].max()
       

        #if peak hourly rain is nan (e.g. there's no data on the day of the landslide, then)

        if pd.isnull(mxhr):
            
            
            idxmxhr = np.nan
            mxhr = np.nan
            idxtrst = np.nan
            htrsttopk = np.nan
            cptrsttopk = np.nan
            htrsttoeod = np.nan
            cptrsttoeod = np.nan
            mwtopk = np.nan
            nnantopk = np.nan
            nnantoeod = np.nan
            

        else:
            
            #find out when the peak occurred
            idxmxhr = ts.loc[lsdate:lsdate+datetime.timedelta(hours = 24)].idxmax()


            #moving window summing, index is last observation in window (eg. 10 am is 8am + 9am + 10 am)
            mw = ts.rolling(nhdry).sum() 

            #subset moving window time series up to the peak 
            mwtopk = mw.loc[:idxmxhr]

            # find where the nhdry hourly sum is less than 0.01 mm (essentially 0, but catch very small values)
            #, find the nearest index to the peak point (iloc type index)
            ix = mwtopk[mwtopk<0.01].index.get_indexer([idxmxhr], method = 'pad')[0]

            #if we can't find a dry enough period in the time series:
            if ix == -1:

                idxtrst = np.nan
                htrsttopk = np.nan
                cptrsttopk = np.nan
                htrsttoeod = np.nan
                cptrsttoeod = np.nan
                mwtopk = np.nan
                nnantopk = np.nan
                nnantoeod = np.nan
                

            else:

                #get the time stamp index of the start of the triggering rainfall 
                idxtrst = mwtopk[mwtopk<0.01].index[ix] + datetime.timedelta(hours = 1)

                #how long did it rain between the start of the triggering rain and the peak (inclusive)? 

                htrsttopk = ts.loc[idxtrst:idxmxhr].count()

                #how much did it rain between the start of the triggering rain and the peak (inclusive)?
                cptrsttopk = ts.loc[idxtrst:idxmxhr].sum()

                #how long did it rain between the start of the triggering rain and the end of the landslide day? 

                htrsttoeod = ts.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].count()

                #how much did it rain between the start of the triggering rain and the end of the landslide day?
                cptrsttoeod = ts.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].sum()
                
                
                #check what percent of the original time series leading up to the peak is nans
                
                nnantopk = ts_raw.loc[idxtrst:idxmxhr].isna().sum()
                
                #check what percent of the original time series leading up to the end of day is nans
                
                nnantoeod = ts_raw.loc[idxtrst:lsdate+datetime.timedelta(hours = 24)].isna().sum()
                             
                
                
                

    return [idxmxhr, 
            mxhr, 
            idxtrst, 
            htrsttopk, 
            cptrsttopk, 
            htrsttoeod, 
            cptrsttoeod, 
            mwtopk, 
            nnantopk,
            nnantoeod]



In [ ]:
def get_antecedent(ts, idxdt, h):
    
    """
    get cumulative antecedent precipitation before some hour (e.g. peak precip on landslide day 
    or before start of triggering rainfall)
    
    ts = time series
    idxdt = datetime that you want antecedent for
    h = number of antecedent hours
    
    """
    #for the case that there no start to the triggering rainfall was found
    if pd.isnull(idxdt):
        ante = np.nan
        
    #for the case that the time series is empty
    elif len(ts) == 0:
        ante = np.nan
    
    else:
        
        ante = ts.loc[idxdt - datetime.timedelta(hours = h):idxdt].sum()
    
    return ante

In [ ]:
def get_rainmetrics_medellin(lspt, medellin_rain):
    
    """
    get info about triggering, event, and antecedent rainfall for a landslide
    
    lspt - a dataframe row from lsdata
    
    """
    print(lspt.name)
    
    [ts, ts_fill] = prep_ts_medellin(lspt, medellin_rain, daysbefore = 90, daysafter = 1)
    
    if ts is None: 
        
        rd = None
        
    else:
    
        lsdate = lspt['date_local_midnight_utc']

        #then, get triggering rainfall metrics

        #define triggering event as 3 hours dry to peak or end of day
        [idxmxhr, mxhr, idxtrst, 
         htrsttopk, cptrsttopk, htrsttoeod, cptrsttoeod, _, nnantrsttopk, nnantrsttoeod] = triggering_rain(nhdry = 3,
                                                                                                   ts = ts_fill,
                                                                                                    ts_raw = ts,
                                                                                                   lsdate = lsdate)

        #get event metrics 

        #define event as 48 hours dry to peak or end of day
        [_, _, idxest, 
         hesttopk, cpesttopk, hesttoeod, cpesttoeod, _, nnanesttopk, nnanesttoeod] = triggering_rain(nhdry = 48,
                                                                                ts = ts_fill,
                                                                                ts_raw = ts,
                                                                                lsdate = lsdate)


        #get antecedent metrics 

        #24 hours before start of triggering rain 

        ante24htrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24)

        #7 day before start of triggering rain 
        ante7dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*7)

        #14 day before start of triggering rain 
        ante14dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*14)

        #21 day before start of triggering rain 
        ante21dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*21)

        #28 day before start of triggering rain
        ante28dtrst = get_antecedent(ts = ts_fill, idxdt = idxtrst, h = 24*28)


        #get estimated exceedance probabilities of the hourly maximum intensity

       # mxhrexprob = get_exceedanceprob(ts, mxhr, 10) #minimum 10 years on record


        #put everything into a dictionary to return 


        rd = {"event_id":lspt.name, 
             "tr_start":idxtrst,
             "tr_tpk":idxmxhr,
             "tr_ppk":mxhr,
             "tr_htopk":htrsttopk, 
             "tr_cptopk":cptrsttopk, 
             "tr_htoeod":htrsttoeod, 
             "tr_cptoeod":cptrsttoeod,
             "tr_nnantopk": nnantrsttopk, 
             "tr_nnan_toeod": nnantrsttoeod,
             "e_start":idxest, 
             "e_htopk":hesttopk, 
             "e_cptopk":cpesttopk, 
             "e_htoeod":hesttoeod, 
             "e_cptoeod":cpesttoeod,
             "e_nnantopk":nnanesttopk,
             "e_nnantoeod":nnanesttoeod,               
             "tr_ante24h":ante24htrst, 
             "tr_ante7d":ante7dtrst, 
             "tr_ante14d":ante14dtrst, 
             "tr_ante21d":ante21dtrst, 
             "tr_ante28d":ante28dtrst}

    print(lspt.name)

    return rd

Get the gauges within a 25 km radius of the landslide

In [ ]:
for l in range(len(medellin_landslides)):
       
    lspt = medellin_landslides.iloc[l]
    
    tempdf = get_medellin_stations(lspt, 
                medellin_stations,
                buffer_dist = 25000)
    
    if l == 0:
        
        gageinfo = tempdf.copy()
        
    else:
        
        gageinfo = pd.concat([gageinfo, tempdf], axis = 0)
    
    print('end {}/{}'.format(l, len(medellin_landslides)))


In [ ]:
#join to landslides dataframe (right join)


medellin_ls_gages = medellin_landslides.merge(gageinfo, 
                           how = 'right',
                            left_index = True,
                           right_on = 'lsidx')

get rain metrics for each landslide/gauge combination

In [ ]:
medellin_ls_gages['rain_metrics'] = medellin_ls_gages.apply(lambda lspt:get_rainmetrics_medellin(lspt, medellin_rain), 
                                             axis = 1)


In [ ]:
joindf = pd.DataFrame(list(medellin_ls_gages['rain_metrics'].values))

lsdata_medellin_rain = pd.concat([medellin_ls_gages, 
             joindf.set_index(medellin_ls_gages.index)], 
             axis = 1)

lsdata_medellin_rain.drop(['rain_metrics'], axis = 1, inplace = True)


#strip city names of white space, special characters, etc for R

lsdata_medellin_rain['city'] = lsdata_medellin_rain.apply(lambda row:''.join(e for e in row['UC_NM_MN'] if e.isalnum()), 
                                       axis = 1)

In [ ]:
#rename some columns to align with gsdr format to be able to concatenate later

lsdata_medellin_rain.rename(columns = {'latitud':'Latitude', 
                                      'longitud':'Longitude', 
                                      'CODIGO':'OriginalID'}, 
                           inplace = True)


In [ ]:
#add some fields of nans to be able to concatenate to gsdr later

lsdata_medellin_rain['Folder'] = np.nan 
lsdata_medellin_rain['NewID'] = np.nan
lsdata_medellin_rain['Recordlength(hours)'] = np.nan
lsdata_medellin_rain['Recordlength(years)'] = np.nan
lsdata_medellin_rain['StartDate'] = np.nan
lsdata_medellin_rain['EndDate'] = np.nan
lsdata_medellin_rain['Missingdata(%)'] = np.nan
lsdata_medellin_rain['coverage'] = np.nan
lsdata_medellin_rain['precip_source'] = 'IDEAM'



In [ ]:
lsdata_medellin_rain_save = lsdata_medellin_rain.loc[:,['inventory', 'src_index', 'inventory_id', 'inventory_id_name', 'lsidx',
       'ID_HDC_G0', 'UC_NM_MN', 'date_local_midnight_utc', 'Folder',
       'OriginalID', 'NewID', 'Latitude', 'Longitude', 'Recordlength(hours)',
       'Recordlength(years)', 'StartDate', 'EndDate', 'Missingdata(%)',
       'geometry_y', 'flag', 'coverage', 'station_dist', 'event_id',
       'tr_start', 'tr_tpk', 'tr_ppk', 'tr_htopk', 'tr_cptopk', 'tr_htoeod',
       'tr_cptoeod', 'tr_nnantopk', 'tr_nnan_toeod', 'e_start', 'e_htopk',
       'e_cptopk', 'e_htoeod', 'e_cptoeod', 'e_nnantopk', 'e_nnantoeod',
       'tr_ante24h', 'tr_ante7d', 'tr_ante14d', 'tr_ante21d', 'tr_ante28d',
       'city', 'precip_source']]

In [ ]:
lsdata_medellin_rain_save.to_csv(resultsdir + 'lsdata_medellin_rain.csv')

In [ ]:
lsdata_medellin_rain_save = pd.read_csv(resultsdir + 'lsdata_medellin_rain.csv')

### Extract annual maxima for Medellin

In [ ]:
medellin_rain.columns

In [ ]:
def prep_whole_ts_medellin(station):
    
    """
    Function to read the precipitation time series from a gauge, convert it to UTC,
    interpolate missing values
    
    station = row of a dataframe that contains columns: 
        - 'Folder' - folder containing the data (e.g. US.zip)
        - 'OriginalID' - name of station
    gsdrdir = directory containing the GSDR subfolders
    
    
    Returns: 
    ts = original time series
    ts_fill = original time series, interpolated   
    
    """
            
    ts = medellin_rain[medellin_rain['CodigoEstacion'] == station]['Valor']
    

    #already in UTC

    #sort index, just in case for some reason it wasn't already in sorted, or pandas thinks it isn't
    ts.sort_index(axis = 0, inplace = True)
    
    #interpolate to fill nans in the time series using the pandas built in interpolation method

    ts_fill = ts.interpolate(method = 'time')

    #remove any duplicates by keeping the first duplicated entry only
   
    ts_fill = ts_fill[~ts_fill.index.duplicated()]

    return ts, ts_fill

In [ ]:
uniquestations = lsdata_medellin_rain_save['OriginalID'].unique() #stations with a landslide associated with it

durations = [1, 3, 6, 12, 24, 48, 100, 200, 500, 1000] #durations to extract block maxima for

In [ ]:
with warnings.catch_warnings(record = True):
#loop through all stations associated with a landslide and extract annual block maxima at a range of durations

    #loop over stations

    for s in range(len(uniquestations)):
        
        print(s)
        
        station = lsdata_medellin_rain_save[lsdata_medellin_rain_save['OriginalID'] == uniquestations[s]].iloc[0][['ID_HDC_G0', 
                                                                                          'UC_NM_MN', 
                                                                                          'Folder', 
                                                                                          'OriginalID', 
                                                                                           'NewID', 
                                                                                          'Latitude',
                                                                                          'Longitude',
                                                                                          'Recordlength(hours)',
                                                                                          'Recordlength(years)',
                                                                                          'StartDate',
                                                                                          'EndDate',
                                                                                          'Missingdata(%)',
                                                                                          'geometry_y']]
        

        #get time series

        ts, ts_fill = prep_whole_ts_medellin(uniquestations[s])

        #extract block maxima at a range of durations

        for d in range(len(durations)):

           #moving window on the raw time series
            mw = ts.rolling(durations[d]).sum() #right index is sum of previous d observations. 
            #if there are nans within the window, returns nan

            ann_block_max = mw.resample('Y').max() #take annual max. ignores nans (could be missing data, 
            #will still get max)


            #moving window on the interpolated time series

            mw_fill = ts_fill.rolling(durations[d]).sum() #right index is sum of previous d observations. 
            #if there are nans within the window, returns nan

            ann_block_max_fill = mw_fill.resample('Y').max() #take annual max. ignores nans (could be missing data, 
            #will still get max)
            
            block_max_df = pd.DataFrame(ann_block_max.values, index = ann_block_max.index, columns = ['raw_block_max'])
            #block_max_df = pd.DataFrame(ann_block_max, columns = ['raw_block_max'])
            block_max_df['fill_block_max'] = ann_block_max_fill.values

            #record the number of non-nan observations in the moving window (all other hours in the year are then nan)
            block_max_df['raw_notnainmw'] = mw.resample('Y').count().values

            #record the number of non-nan observations in the raw time series (all other hours in the year are then nan)
            block_max_df['raw_notnaints'] = ts.resample('Y').count().values


            block_max_df['duration(h)'] = durations[d]
            block_max_df['year'] = block_max_df.index.year


            if d == 0:

                stationdf = block_max_df.copy() 



            else:

                stationdf = pd.concat([stationdf, block_max_df])


        #assign information about the station
        stationinfo = pd.DataFrame([station])

        stationinfo = stationinfo.loc[stationinfo.index.repeat(len(stationdf))]

        stationinfo.set_index(stationdf.index, inplace = True)

        stationdf = pd.concat([stationinfo, stationdf], axis = 1)

        if s == 0: 

            maindf = stationdf.copy()

        else: 

            maindf = pd.concat([maindf, stationdf])
            
        print(s)

In [ ]:
maindf.to_csv(resultsdir + 'annual_block_maxima_medellin.csv')